In [1]:
# Import dependencies
import requests
import os
import pprint
from datetime import datetime,timedelta
import json
import pandas as pd
from dateutil.parser import parse
import itertools
from itertools import permutations
import tqdm 
from tqdm import tqdm
import csv


In [2]:
# Set params for query
pax = input(' # of adults will be traveling?')
children = input(' # Children traveling with you?')
p0 = input(' Starting place') 
startdate = input('Trip start date (mm/dd/yyyy)')
destinations = input('List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3,Paris,6)')
#destinations.replace(" ", "")
destinations1 = list(destinations.split(','))




 # of adults will be traveling?1
 # Children traveling with you?0
 Starting placemiami
Trip start date (mm/dd/yyyy)12/01/2019
List the cities do you want to travel to? And how many days per city? ex(London,4,Madrid,3,Paris,6)madrid,4,london,3,paris,5


In [3]:
p1=(str(destinations1[0]),int(destinations1[1]))
p2 = (str(destinations1[2]), int(destinations1[3]))
p3 = (str(destinations1[4]), int(destinations1[5]))


def changeorder(res,p1,p2,p3,startdate,p0,children,pax):
    

    start_date = parse(startdate)
    days_p1=p1[1]
    days_p2=p2[1]
    days_p3=p3[1]
    start_p1 = start_date + timedelta(days=days_p1)
    start_p2 = start_p1 + timedelta(days=days_p2)
    start_p3 = start_p2 + timedelta(days=days_p3)
    url = "https://apidojo-hipmunk-v1.p.rapidapi.com/flights/create-session"

    querystring = {"from1":p1[0],"from2":p2[0],"from3":p3[0],"to1":p2[0],"to2":p3[0],"to3":p0,"date1":start_p1,"date2":start_p2,"date3":start_p3,"infants_lap":"0","children":children,"seniors":"0","country":"US","from0":p0,"to0":p1[0],"date0":startdate,"pax":pax,"cabin":"Coach"}
    headers = {
        'x-rapidapi-host': "apidojo-hipmunk-v1.p.rapidapi.com",
        'x-rapidapi-key': "c36e6ce6fcmsh7fe55ace5e9f40fp10fdc1jsn690a20bbc0ef"
        }

    response = requests.request("GET", url, headers=headers, params=querystring).json()

    #does stuff with response
    idnums=[]
    agonies=[]
    prices=[]
    routingid=[]
    

    for key,value in response['itins'].items():
        idnum=key
        agony=value['agony']
        price=value['price']
        routing_id=value['routing_idens']
        idnums.append(idnum)
        agonies.append(agony)
        prices.append(price)
        routingid.append(routing_id)
    df=pd.DataFrame()
    df['price']=prices
    df['agonies']=agonies
    df['id']=idnums
    df['routing']=routingid
    
    
    
    


    # Print DF with values
    df = df.sort_values(['price','agonies'], ascending=[True,True])
    df = df[:1]

    res.append([(p1,p2,p3),df])



    #p1,p2,p3 = p2,p3,p1
    return res
    
    

In [4]:
alist = []
listA = [p1, p2, p3]
perm = itertools.permutations(listA) 
  
for i in list(perm): 
    alist.append(i)
alist
    
    
    

[(('madrid', 4), ('london', 3), ('paris', 5)),
 (('madrid', 4), ('paris', 5), ('london', 3)),
 (('london', 3), ('madrid', 4), ('paris', 5)),
 (('london', 3), ('paris', 5), ('madrid', 4)),
 (('paris', 5), ('madrid', 4), ('london', 3)),
 (('paris', 5), ('london', 3), ('madrid', 4))]

In [5]:
print('Sorry, this may take a while...')
res=[]
for i in tqdm(alist):
    p1,p2,p3=i[0],i[1],i[2]
    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)
    
res=[]
for i in tqdm(alist):
    p1,p2,p3=i[0],i[1],i[2]
    res=changeorder(res,p1,p2,p3,startdate,p0,children,pax)    
    
print('Search 100% completed')   
res

  0%|          | 0/6 [00:00<?, ?it/s]

Sorry, this may take a while...


100%|██████████| 6/6 [01:54<00:00, 19.15s/it]

Search 100% completed


[[(('madrid', 4), ('london', 3), ('paris', 5)),
        price      agonies                                id  \
  4134  756.0  1892.829071  95b1d37a22c0ee37b03ffe6912fc4bee   
  
                                                  routing  
  4134  [458dfcf2fb8507a7fc373035c2f493c1, b48af660829...  ],
 [(('madrid', 4), ('paris', 5), ('london', 3)),
        price     agonies                                id  \
  3167  921.0  2266.81758  687b5badf9925da67bcf8848d6c99373   
  
                                                  routing  
  3167  [458dfcf2fb8507a7fc373035c2f493c1, a0f4a890ef8...  ],
 [(('london', 3), ('madrid', 4), ('paris', 5)),
        price      agonies                                id  \
  1885  836.0  2089.246992  31d0d781689ff8c075dda41187eb75e8   
  
                                                  routing  
  1885  [cf93ba9848d39a0b0df94cae99934983, 8d08cc6c4be...  ],
 [(('london', 3), ('paris', 5), ('madrid', 4)),
       price      agonies                          

In [6]:
df1=pd.DataFrame(res[0][1])
df2=pd.DataFrame(res[1][1])
df3=pd.DataFrame(res[2][1])
df4=pd.DataFrame(res[3][1])
df5=pd.DataFrame(res[4][1])
df6=pd.DataFrame(res[5][1])


In [7]:
#Joining all results into a single DF:

#Option 1
first_choice = df1['routing'].astype(str)
df1new = first_choice.str.split(", ",expand=True)
firstchoice_merged = df1.merge(df1new, how='outer', left_index=True, right_index=True)
firstchoice_merged = firstchoice_merged.drop(columns=['routing'])

#Option 2
second_choice = df2['routing'].astype(str)
df2new = second_choice.str.split(", ",expand=True)
secondchoice_merged = df2.merge(df2new, how='outer', left_index=True, right_index=True)
secondchoice_merged = secondchoice_merged.drop(columns=['routing'])

#Option 3
third_choice = df3['routing'].astype(str)
df3new = third_choice.str.split(", ",expand=True)
thirdchoice_merged = df3.merge(df3new, how='outer', left_index=True, right_index=True)
thirdchoice_merged = thirdchoice_merged.drop(columns=['routing'])

#Option 4
fourth_choice = df4['routing'].astype(str)
df4new = fourth_choice.str.split(", ",expand=True)
fourthchoice_merged = df4.merge(df4new, how='outer', left_index=True, right_index=True)
fourthchoice_merged = fourthchoice_merged.drop(columns=['routing'])

#Option 5
fifth_choice = df5['routing'].astype(str)
df5new = fifth_choice.str.split(", ",expand=True)
fifthchoice_merged = df5.merge(df5new, how='outer', left_index=True, right_index=True)
fifthchoice_merged = fifthchoice_merged.drop(columns=['routing'])

#Option 6
sixth_choice = df6['routing'].astype(str)
df6new = sixth_choice.str.split(", ",expand=True)
sixthchoice_merged = df6.merge(df6new, how='outer', left_index=True, right_index=True)
sixthchoice_merged = sixthchoice_merged.drop(columns=['routing'])

bigdata = [firstchoice_merged,secondchoice_merged,thirdchoice_merged,fourthchoice_merged,fifthchoice_merged,sixthchoice_merged]
result = pd.concat(bigdata)
#result_df = result.sort_values(['price','agonies'], ascending=[True,True])

In [8]:
resultuse_df = result.drop(columns=[0,1,2,3])
resultuse_df


,price,agonies,id
4134,756.0,1892.829071,95b1d37a22c0ee37b03ffe6912fc4bee
3167,921.0,2266.817580,687b5badf9925da67bcf8848d6c99373
1885,836.0,2089.246992,31d0d781689ff8c075dda41187eb75e8
340,948.0,2591.407324,4b4457350814fb577ff2e8e1420dcb30
2598,952.0,2247.553815,74f5e47de4ead87b3018c60b7799db2c
5225,907.0,2452.744867,41e88f9b9283309febf909ef2dfd6056


In [9]:
xlist1=[]
ylist1=[]
xlist2=[]
ylist2=[]
xlist3=[]
ylist3=[]

for x,y,z in alist:
    d1,d2 = x
    xlist1.append(d1)
    ylist1.append(d2)
    d3,d4 = y
    xlist2.append(d3)
    ylist2.append(d4)
    d5,d6 = z
    xlist3.append(d5)
    ylist3.append(d6)
print(xlist1)
print(ylist1)



['madrid', 'madrid', 'london', 'london', 'paris', 'paris']
[4, 4, 3, 3, 5, 5]


In [10]:
alist

[(('madrid', 4), ('london', 3), ('paris', 5)),
 (('madrid', 4), ('paris', 5), ('london', 3)),
 (('london', 3), ('madrid', 4), ('paris', 5)),
 (('london', 3), ('paris', 5), ('madrid', 4)),
 (('paris', 5), ('madrid', 4), ('london', 3)),
 (('paris', 5), ('london', 3), ('madrid', 4))]

In [11]:
alist_df = pd.DataFrame(alist)
alist_df
alist_df

,0,1,2
0,"(madrid, 4)","(london, 3)","(paris, 5)"
1,"(madrid, 4)","(paris, 5)","(london, 3)"
2,"(london, 3)","(madrid, 4)","(paris, 5)"
3,"(london, 3)","(paris, 5)","(madrid, 4)"
4,"(paris, 5)","(madrid, 4)","(london, 3)"
5,"(paris, 5)","(london, 3)","(madrid, 4)"


In [12]:
start = parse(startdate)
#start = start_date.strftime("%B %d, %Y")

#Translating the lenght of stay into a date for first stop departure
g=[]
date1=[]
for days1 in ylist1:
    g = start + timedelta(days=days1)
    #g1 = g.strftime("%B %d, %Y")
    date1.append(g)

#Translating the lenght of stay into a date for second stop departure
c=[]
date2=[]
c1=[]
c2=[]
i=[]

for days2 in date1:
    c1.append(days2)

for i in range(6):    
    xi = (c1[i])
    xh = (ylist2[i])
    c2 = xi + timedelta(days=xh)
    date2.append(c2) 
    
#Translating the lenght of stay into a date for third stop departure    
    
c3=[]
date3=[]
c3=[]
c4=[]
i=[]

for days3 in date2:
    c3.append(days3)

for i in range(6):    
    xe = (c3[i])
    xr = (ylist3[i])
    c4 = xe + timedelta(days=xr)
    date3.append(c4)

                               
resultuse_df['Starting_Point'] = p0
resultuse_df['Starting_Date'] = start
resultuse_df['First_Stop'] = xlist1
resultuse_df['Departure_1'] = date1
resultuse_df['Second_Stop'] = xlist2
resultuse_df['Departure_2'] = date2
resultuse_df['Third_Stop'] = xlist3
resultuse_df['Departure_3'] = date3
resultuse_df['End_Point'] = p0
#resultuse_df = resultuse_df.drop[['Starting_Point','Starting_Date']]
df_full = resultuse_df[['price','agonies','Starting_Point','Starting_Date','First_Stop','Departure_1','Second_Stop','Departure_2','Third_Stop','Departure_3','End_Point','id']]

In [13]:
df_fullnew = df_full.rename(columns={"id":'Itinerary_#'})
df_fullnew=df_fullnew.sort_values(['price','agonies'], ascending=[True,True])

In [14]:
df_fullnew

,price,agonies,Starting_Point,Starting_Date,First_Stop,Departure_1,Second_Stop,Departure_2,Third_Stop,Departure_3,End_Point,Itinerary_#
4134,756.0,1892.829071,miami,2019-12-01,madrid,2019-12-05,london,2019-12-08,paris,2019-12-13,miami,95b1d37a22c0ee37b03ffe6912fc4bee
1885,836.0,2089.246992,miami,2019-12-01,london,2019-12-04,madrid,2019-12-08,paris,2019-12-13,miami,31d0d781689ff8c075dda41187eb75e8
5225,907.0,2452.744867,miami,2019-12-01,paris,2019-12-06,london,2019-12-09,madrid,2019-12-13,miami,41e88f9b9283309febf909ef2dfd6056
3167,921.0,2266.817580,miami,2019-12-01,madrid,2019-12-05,paris,2019-12-10,london,2019-12-13,miami,687b5badf9925da67bcf8848d6c99373
340,948.0,2591.407324,miami,2019-12-01,london,2019-12-04,paris,2019-12-09,madrid,2019-12-13,miami,4b4457350814fb577ff2e8e1420dcb30
2598,952.0,2247.553815,miami,2019-12-01,paris,2019-12-06,madrid,2019-12-10,london,2019-12-13,miami,74f5e47de4ead87b3018c60b7799db2c


In [15]:
destination_df = df_fullnew.loc[:,['Starting_Point','First_Stop','Second_Stop','Third_Stop','End_Point']]
destination_df

,Starting_Point,First_Stop,Second_Stop,Third_Stop,End_Point
4134,miami,madrid,london,paris,miami
1885,miami,london,madrid,paris,miami
5225,miami,paris,london,madrid,miami
3167,miami,madrid,paris,london,miami
340,miami,london,paris,madrid,miami
2598,miami,paris,madrid,london,miami


In [16]:
# Create an empty list 
destinations_list =[] 
  
# Iterate over each row 
for index, rows in destination_df.iterrows(): 
    # Create list for the current row 
    my_list =[rows.Starting_Point, rows.First_Stop, rows.Second_Stop, rows.Third_Stop, rows.End_Point] 
      
    # append the list to the final list 
    destinations_list.append(my_list) 
  
# Print the list 
print(destinations_list) 


[['miami', 'madrid', 'london', 'paris', 'miami'], ['miami', 'london', 'madrid', 'paris', 'miami'], ['miami', 'paris', 'london', 'madrid', 'miami'], ['miami', 'madrid', 'paris', 'london', 'miami'], ['miami', 'london', 'paris', 'madrid', 'miami'], ['miami', 'paris', 'madrid', 'london', 'miami']]


In [23]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="specify_your_app_name_here")

from geopy.extra.rate_limiter import RateLimiter
geocode = RateLimiter(geolocator.geocode)
destination_df['location'] = destination_df['Starting_Point'].apply(geocode)
destination_df['point'] = destination_df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
destination_df['location2'] = destination_df['First_Stop'].apply(geocode)
destination_df['point2'] = destination_df['location2'].apply(lambda loc: tuple(loc.point) if loc else None)
destination_df['location3'] = destination_df['Second_Stop'].apply(geocode)
destination_df['point3'] = destination_df['location3'].apply(lambda loc: tuple(loc.point) if loc else None)
destination_df['location4'] = destination_df['Third_Stop'].apply(geocode)
destination_df['point4'] = destination_df['location4'].apply(lambda loc: tuple(loc.point) if loc else None)

In [24]:
destination_df

,Starting_Point,First_Stop,Second_Stop,Third_Stop,End_Point,location,point,location2,point2,location3,point3,location4,point4
4134,miami,madrid,london,paris,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)"
1885,miami,london,madrid,paris,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)"
5225,miami,paris,london,madrid,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)"
3167,miami,madrid,paris,london,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)"
340,miami,london,paris,madrid,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)"
2598,miami,paris,madrid,london,miami,"(Miami, Miami-Dade County, Florida, United Sta...","(25.7742658, -80.1936589, 0.0)","(Paris, Île-de-France, France, (48.8566101, 2....","(48.8566101, 2.3514992, 0.0)","(Madrid, Área metropolitana de Madrid y Corred...","(40.4167047, -3.7035825, 0.0)","(London, Greater London, England, SW1A 2DX, Un...","(51.5073219, -0.1276474, 0.0)"


In [27]:
coorddesti_df = destination_df[['point','point2','point3','point4','point']]
coorddesti_df

,point,point2,point3,point4,point
4134,"(25.7742658, -80.1936589, 0.0)","(40.4167047, -3.7035825, 0.0)","(51.5073219, -0.1276474, 0.0)","(48.8566101, 2.3514992, 0.0)","(25.7742658, -80.1936589, 0.0)"
1885,"(25.7742658, -80.1936589, 0.0)","(51.5073219, -0.1276474, 0.0)","(40.4167047, -3.7035825, 0.0)","(48.8566101, 2.3514992, 0.0)","(25.7742658, -80.1936589, 0.0)"
5225,"(25.7742658, -80.1936589, 0.0)","(48.8566101, 2.3514992, 0.0)","(51.5073219, -0.1276474, 0.0)","(40.4167047, -3.7035825, 0.0)","(25.7742658, -80.1936589, 0.0)"
3167,"(25.7742658, -80.1936589, 0.0)","(40.4167047, -3.7035825, 0.0)","(48.8566101, 2.3514992, 0.0)","(51.5073219, -0.1276474, 0.0)","(25.7742658, -80.1936589, 0.0)"
340,"(25.7742658, -80.1936589, 0.0)","(51.5073219, -0.1276474, 0.0)","(48.8566101, 2.3514992, 0.0)","(40.4167047, -3.7035825, 0.0)","(25.7742658, -80.1936589, 0.0)"
2598,"(25.7742658, -80.1936589, 0.0)","(48.8566101, 2.3514992, 0.0)","(40.4167047, -3.7035825, 0.0)","(51.5073219, -0.1276474, 0.0)","(25.7742658, -80.1936589, 0.0)"


In [28]:
mergedDf = coorddesti_df.merge(df_fullnew, left_index=True, right_index=True)
mergedDf

,point,point2,point3,point4,point,price,agonies,Starting_Point,Starting_Date,First_Stop,Departure_1,Second_Stop,Departure_2,Third_Stop,Departure_3,End_Point,Itinerary_#
4134,"(25.7742658, -80.1936589, 0.0)","(40.4167047, -3.7035825, 0.0)","(51.5073219, -0.1276474, 0.0)","(48.8566101, 2.3514992, 0.0)","(25.7742658, -80.1936589, 0.0)",756.0,1892.829071,miami,2019-12-01,madrid,2019-12-05,london,2019-12-08,paris,2019-12-13,miami,95b1d37a22c0ee37b03ffe6912fc4bee
1885,"(25.7742658, -80.1936589, 0.0)","(51.5073219, -0.1276474, 0.0)","(40.4167047, -3.7035825, 0.0)","(48.8566101, 2.3514992, 0.0)","(25.7742658, -80.1936589, 0.0)",836.0,2089.246992,miami,2019-12-01,london,2019-12-04,madrid,2019-12-08,paris,2019-12-13,miami,31d0d781689ff8c075dda41187eb75e8
5225,"(25.7742658, -80.1936589, 0.0)","(48.8566101, 2.3514992, 0.0)","(51.5073219, -0.1276474, 0.0)","(40.4167047, -3.7035825, 0.0)","(25.7742658, -80.1936589, 0.0)",907.0,2452.744867,miami,2019-12-01,paris,2019-12-06,london,2019-12-09,madrid,2019-12-13,miami,41e88f9b9283309febf909ef2dfd6056
3167,"(25.7742658, -80.1936589, 0.0)","(40.4167047, -3.7035825, 0.0)","(48.8566101, 2.3514992, 0.0)","(51.5073219, -0.1276474, 0.0)","(25.7742658, -80.1936589, 0.0)",921.0,2266.817580,miami,2019-12-01,madrid,2019-12-05,paris,2019-12-10,london,2019-12-13,miami,687b5badf9925da67bcf8848d6c99373
340,"(25.7742658, -80.1936589, 0.0)","(51.5073219, -0.1276474, 0.0)","(48.8566101, 2.3514992, 0.0)","(40.4167047, -3.7035825, 0.0)","(25.7742658, -80.1936589, 0.0)",948.0,2591.407324,miami,2019-12-01,london,2019-12-04,paris,2019-12-09,madrid,2019-12-13,miami,4b4457350814fb577ff2e8e1420dcb30
2598,"(25.7742658, -80.1936589, 0.0)","(48.8566101, 2.3514992, 0.0)","(40.4167047, -3.7035825, 0.0)","(51.5073219, -0.1276474, 0.0)","(25.7742658, -80.1936589, 0.0)",952.0,2247.553815,miami,2019-12-01,paris,2019-12-06,madrid,2019-12-10,london,2019-12-13,miami,74f5e47de4ead87b3018c60b7799db2c


In [33]:
import plotly.graph_objects as go

fig = go.Figure(go.Scattermapbox(
    mode = "markers+lines",
    
    marker = {'size': 5}))

fig.add_trace(go.Scattermapbox(
mode = "markers+lines",
lon = [-80.1936589, -3.7035825,-0.1276474,2.3514992,-80.1936589],
lat = [25.7742658, 40.4167047, 51.5073219,48.8566101,25.7742658],
marker = {'size': 5}))

fig.update_layout(
    margin ={'l':0,'t':0,'b':0,'r':0},
    mapbox = {
        'center': {'lon': 10, 'lat': 10},
        'style': "stamen-terrain",
        'center': {'lon': -20, 'lat': -20},
        'zoom': 1})
fig.show()